# Video 2 Recipe

Programa capaz de extraer recetas de cocina a partir de la transcripción del audio de un video de cocina en Youtube. Útil para aquellas recetas que no cuentan con una transcripción en la descripción.

## Imports

In [1]:
import os
import sys

# pip install --upgrade youtube-dl
import youtube_dl as ydl

# pip install webvtt-py
import webvtt

# Para capturar stdout como un string
from io import StringIO

## Video a Procesar

In [2]:
# URL del video del que se desea extraer una receta
video_url = "https://www.youtube.com/watch?v=Vr-o01qiRYI"

## Chequeo Subtítulos

Se solicitan los subtítulos de un video. Si el video no tiene subtítulos oficiales, la bandera "has_subs" se retorna como False.

In [3]:
# Reemplaza el stdout normal por uno custom llamado "mystdout"
old_stdout = sys.stdout
sys.stdout = temp_stdout = StringIO()

# Opciones de descarga:
# - Listar los subtítulos disponibles
# - No descargar el video
download_options = {
    'listsubtitles': True,
    "skip_download": True
}

# Se realiza el request. El progreso del programa es guardado
# en la variable "mystdout"
with ydl.YoutubeDL(download_options) as video:
    video.download([video_url])

# Se vuelve a poner "stdout" como logger
sys.stdout = old_stdout

# Se guardan los logs generados en una variable
ydl_logs = temp_stdout.getvalue()

# Si los logs contienen el string "has no subtitles" se
# setea una variable como "False" para indicar esto.
if "has no subtitles" in ydl_logs:
    has_subtitles = False
    print("El video no tiene subtítulos.")
else:
    has_subtitles = True
    print("El video tiene subtítulos")

El video no tiene subtítulos.


## Creación Corpus

In [4]:
# Opciones de descarga:
# - Descargar todos los archivos como "source"
# - Descargar los subtítulos ya dados por el video
# - Obviar la descarga del video como tal.
# - Se descargan los subtítulos del video (oficiales si existen, automáticos si no los hay)
download_options = {
    'outtmpl': './media/subs',
    'writesubtitles': True,
    'writeautomaticsub': True,
    "skip_download": True
}

# Se realiza la descarga
with ydl.YoutubeDL(download_options) as video:
    video.download([video_url])

# Se listan todos los archivos de la carpeta "media"
files = os.listdir("./media")

# Se revisa si algún archivo contiene "subs" en su nombre
# de ser así, se extrae su path.
for file in files:
    if "subs" in file:
        subtitles_path = file
        break

# ======================
# SUBTÍTULOS MANUALES
# ======================

if has_subtitles:
    
    # Corpus del video
    video_corpus = ""

    # Se extrae el texto del archivo .vtt  
    for caption in webvtt.read("./media/" + subtitles_path):

        # Se eliminan:
        # - Hard spaces (&nbsp;)
        # - Newlines (\n)
        # - Leading and trailing spaces
        caption = caption.text.replace("&nbsp;", " ")
        caption = caption.replace("\n", "")
        caption = caption.strip()

        # Se agrega el string limpio al corpus
        video_corpus = video_corpus + " " + caption

# ======================
# SUBTÍTULOS AUTOMÁTICOS
# ======================

else:
    
    # Se inicializa la string de captions y la lista de snippets
    # a incluir en el corpus final
    previous_caption = ""
    caption_snippets = []

    for i, caption in enumerate(webvtt.read("./media/" + subtitles_path)):

        caption = caption.text.replace("&nbsp;", " ")
        caption = caption.replace("\n", "")
        caption = caption.strip()

        # Se agrega siempre la primera caption a los snippets
        if i == 0:
            caption_snippets.append(caption)

        # Si la caption anterior es parte de la nueva caption
        # se agrega la caption anterior a los snippets
        elif previous_caption in caption:
            caption_snippets.append(previous_caption)

        # Se actualiza la caption anterior
        previous_caption = caption

    # Se eliminan los strings repetidos 
    # Se crea un diccionario que utiliza como llaves los strings del corpus
    # dado que un diccionario no puede tener llaves repetidas, elimina las
    # repetidas y las retorna ordenadas. Se puede hacer la misma operación 
    # usando "sets" pero retorna los elementos del corpus desordenados.
    caption_snippets = list(dict.fromkeys(caption_snippets))

    # Se unen todas las strings
    video_corpus = " ".join(caption_snippets)


[youtube] Vr-o01qiRYI: Downloading webpage
[info] Writing video subtitles to: media\subs.en.vtt


## Visualizando Corpus

In [5]:
print(video_corpus)

[Music]  America I know what you're thinking do we really need a recipe for baked potatoes well here in the Test Kitchen we baked over 200 pounds of spuds to discover that very answer and today I'm here with the expert Ellie who's gonna show us why we do need a recipe Bridgette some crazy things are happening in the world with baked potatoes and it has to stop Oh No immediately first we're cooking our potatoes in the microwave not good I've done it not gonna I've done it's right and it cooks unevenly it cooks from the inside out we also cook our potatoes in foil I've done that too and it traps in all of the moisture and it doesn't give us a tasty potato and finally when we do get it in the oven to bake it we let it hang out on the counter forever and there's no fluffiness coming out of that potato at all but their greatest doorstops absolutely today we're gonna do my favorite thing one of the things we love to do in the Test Kitchen we're gonna brine potatoes we're gonna brine potatoes

## BERT para NER (Named Entity Recognition)

Se utiliza el modelo de BERT para predecir o taggear las entidades presentes en una oración. Basado en el ejemplo dado en: https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/

In [12]:
import pandas as pd 
import numpy as np
from tqdm import tqdm, trange

# Se lee el dataset
# Viene de Kaggle: https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus#ner_dataset.csv
dataset = pd.read_csv("ner_dataset.csv", encoding="latin1")

# Un detalle importante de este dataset es que las oraciones están concatenadas verticalmente
# por lo que solo se le asigna el tag de a que oración pertenece a la primera palabra de la oración. 
# Se debe tomar esto en cuenta y rellenar las tags faltantes usando pandas. 
dataset = dataset.fillna(method="ffill")

# Se visualiza el dataset
# Tiene 4 columnas:
# - Oración a la que pertence la palabra
# - Palabra de oración
# - POS: Tipo de sintáxis de una palabra en la oración (Part of Sentence)
# - Tag: Tipo de palabra (semántica)
dataset.head(18)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [26]:
# Se crea una clase para convertir todas las palabras de una oración a una lista
# También se concatenan las "POS" y "Tags" que acompañan a los datos.
class SentenceGetter(object):

    def __init__(self, data):

        # Número de listas "enviadas" o impresas
        self.n_sent = 0

        # Se crean las variables internas de la función
        self.data = data
        self.empty = False

        # Pasos:
        # 1. Se convierten las columnas de "Word", "POS" y "Tags" en listas
        # 2. Las listas se convierten en un iterador de tuplas con tres elementos (Word, POS, Tags)
        # 3. Se convierte el iterador en una lista
        aggregate_func = lambda sen: list(
            zip(sen["Word"].values.tolist(), 
                sen["POS"].values.tolist(), 
                sen["Tag"].values.tolist())
        )

        # Se agrupa la data original de acuerdo a la oración a la que pertenece
        # Para los valores de cada oración, estos se colocan en listas de tuplas
        # como se especificó arriba. Los elementos de la lista se pueden indexar
        # escribiendo: "grouped_data['Sentence 1']" por ejemplo.
        self.grouped_data = self.data.groupby("Sentence #").apply(aggregate_func)

        # Se convierte la serie en una lista
        self.sentence_values = self.grouped_data.to_list()

        # Se extraen únicamente las palabras
        self.sentence = [[word[0] for word in sentence_values] for sentence_values in self.sentence_values]
        self.POS      = [[word[1] for word in sentence_values] for sentence_values in self.sentence_values]
        self.tags     = [[word[2] for word in sentence_values] for sentence_values in self.sentence_values]

    # Rutina para obtener la siguiente oración en la secuencia
    def get_next(self):

        try:   
            # Se retorna el elemento "n_sent" de la lista
            s = self.grouped_data[f"Sentence: {self.n_sent}"]

            # Se incrementa el index utilizado
            self.n_sent += 1
            return s

        except Exception as e:
            return None


# Se crea una instancia de la clase anterior
# (Se procesan los datos del dataset)
sentenceGetter = SentenceGetter(dataset)

# Se extrae la oración 1
sentenceGetter.sentence[0]

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country',
 '.']

Se extraen los valores únicos para las diferentes tags existentes en el dataset.

In [30]:
# Clases únicas (tags) en el dataset
# - geo: Entidad geográfica
# - org: Organización
# - per: Persona
# - gpe: Entidad geopolítica
# - tim: Indicador de tiempo
# - art: Artefacto
# - eve: Evento
# - nat: Fenómeno natural
# Se extraen los valores de la columna "Tag" y luego se eliminan los repetidos
unique_tags = list(set(dataset["Tag"].values))

# Se adiciona una clase única para palabras de padding
unique_tags.append("PAD")

# Vector para mapear cada clase a un index diferente
# (Se le asigna un número a cada clase)
tag_codes = {tag: idx for idx, tag in enumerate(unique_tags)}

Se prepara Pytorch y BERT para ser utilizado

In [37]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Configuraciones (Hiperparámetros)
# Max Length: Largo máximo de una secuencia en número de tokens (BERT soporta hasta 512 tokens)
# Batch Size: Número de muestras pasadas a la vez al algoritmo (32 es una sugerencia del paper de BERT)
max_len = 75
batch_size = 32

# Se intenta configurar la GPU del sistema. Si no lo consigue
# entonces utiliza el CPU y se imprime el nombre del procesador utilizado
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Number of Devices:", torch.cuda.device_count())
print("Device Name:", torch.cuda.get_device_name(0))

# BERT ya viene con un tokenizador pre-entrenado y con un vocabulario definido
# - Se carga el modelo más pequeño: "bert-base-uncased"
# - Se llevan todas las letras a minúsculas: do_lower_case = True
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Number of Devices: 1
Device Name: NVIDIA GeForce GTX 960M


Se tokenizan las palabras de cada oración

In [44]:
# Se tokenizan todas las oraciones. Debido a que BERT está basado en el tokenizador
# de Wordpiece, el mismo va a separar tokens en subpalabras (cada una con su propio
# token). Por ejemplo, "gunships" lo va a separar en "guns" y "##hips". Esto se debería
# de solucionar utilizando una estructura de datos especial basada en "label spans",
# sin embargo, aquí se hace de manera explícita.
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    # Para cada pareja de oración y labels
    for word, label in zip(sentence, text_labels):

        # Se tokeniza la palabra y se cuenta el número de subpalabras en las que se rompe
        tokenized_word = tokenizer.tokenize(word)
        num_subwords = len(tokenized_word)

        # Se incluye la palabra tokenizada en la lista de palabras tokenizadas
        tokenized_sentence.extend(tokenized_word)

        # Se agrega la misma label a todas las subpalabras creadas
        # (Sse multiplica por "num_subwords" la label actual)
        labels.extend([label] * num_subwords)

    return tokenized_sentence, labels

# Se obtiene:
# - Lista de palabras en la oración (sentences)
# - Lista de tags para cada palabra (labels)
sentences = sentenceGetter.sentence
labels = sentenceGetter.tags

# Se aplica la rutina anterior para todas las oraciones
tokenized_texts_and_labels = [tokenize_and_preserve_labels(sens, labs) for sens, labs in zip(sentences, labels)]

# Se extraen las palabras tokenizadas y las labels "multiplicadas"
tokenized_texts  = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
tokenized_labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

Se hace padding a las secuencias para que encajen con el largo máximo especificado arriba (max length)

In [48]:
# Se cortan y se paddean los tokens 
# 1. Se convierten los tokens a "IDs"
# 2. El largo máximo es el previamente especificado
# 3. Se truncan tokens muy largos y se paddean los valores muy cortos con 0s
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", 
                          value=0.0,
                          truncating="post", padding="post")

# Se padean las secuencias las tags
# 1. Se obtienen los IDs de las clases de cada label
# 2. Se cortan al largo deseado
# 3. Se truncan valores largos y se paddean valores con "PAD".
tags = pad_sequences([[tag_codes.get(label) for label in sentence_labels] for sentence_labels in labels],
                     maxlen=max_len, dtype="long", 
                     value=tag_codes["PAD"],
                     truncating="post",  padding="post")

# BERT soporta algo denominado "attention masks", lo cual le permite
# ignorar padding en una secuencia. Se crean máscaras para ignorar el padding.
# 1. Se obtienen los IDs para una oración
# 2. Se chequea si cada ID es diferente de 0.
# 3. Si es igual a 0, se retorna False y se convierte a 0.0. Los demás valores se convierten en 1.0
attention_masks = [[float(id != 0.0) for id in input_id] for input_id in input_ids]

Se crea el train-test split y se adaptan los valores a elementos utilizables por PyTorch

In [50]:
# Split del dataset para utilizar el 10% para validación
train_ids, valid_ids, train_tags, valid_tags = train_test_split(input_ids, tags, random_state=6969, test_size=0.1)

# Se realiza el mismo split para las máscaras de atención
train_masks, valid_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=6969, test_size=0.1)

# Se convierte el dataset en tensores de PyTorch
train_ids   = torch.tensor(train_ids)
train_tags  = torch.tensor(train_tags)
train_masks = torch.tensor(train_masks)
valid_ids   = torch.tensor(valid_ids)
valid_tags  = torch.tensor(valid_tags)
valid_masks = torch.tensor(valid_masks)

# Se convierten los datos de entrenamiento y de validación 
# a datos utilizables por Pytorch
# - Se define un dataset
# - Se crea un "mezclador" de datos (RandomSampler)
# - Se crea un "cargador" de datos
train_dataset    = TensorDataset(train_ids, train_masks, train_tags)
train_sampler    = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

valid_dataset    = TensorDataset(valid_ids, valid_masks, valid_tags)
valid_sampler    = RandomSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=batch_size)

Setup de BERT para realizar finetuning. Si se carga el modelo pre-entrenado para re-entrenarlo, los comandos van a retornar un mensaje de error. Este puede ser ignorado si el modelo aún no se ha entrenado.

In [52]:
import transformers
from transformers import BertForTokenClassification, AdamW
from transformers import get_linear_schedule_with_warmup

# El paquete de "transformers" provee una clase de "BertForTokenClassification", la cual
# consiste de un modelo para "fine-tuning" que agrega un clasificador de tokens al modelo
# base de BERT. El clasificador consiste de una capa lineal que toma como input el último
# estado oculto de la secuencia. A continuación se carga el modelo pre-entrenado:
# - Se utiliza el modelo base para palabras en minúscula
# - Se indica el número de labels a predecir
# - No se sacan las máscaras de atención ni los estados ocultos
model = BertForTokenClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(tag_codes),
    output_attentions = False,
    output_hidden_states = False
)

# Se alimentan los parámetros del modelo a la GPU
model.cuda()

# Se hace el setup del optimizador (ADAM) y se añaden los parámetros que debería de
# actualizar PyTorch. También se agrega "weight decay" para regularizar las matrices
# principales de pesos. Si no se tienen suficientes recursos, se puede setear
# full_finetuning a False y así se congelan los pesos de BERT. 
full_finetuning = True

if full_finetuning:

    # Parámetros a optimizar
    param_optimizer = list(model.named_parameters())

    # Valores a los que no se les aplicará decay
    no_decay = ["bias", "gamma", "beta"]

    # Se setea el decay para cada grupo de parámetros
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if     any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]

else:
    # Solo se optimizan los parámetros 
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [
        {"params": [p for n,p in param_optimizer]}
    ]

# Se configura el optimizador a utilizar
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5, eps=1e-8)

# Hiperparámetros de entrenamiento
epochs = 3
max_grad_norm = 1.0

# Número total de pasos de entrenamiento:
# Número de batches * número de epochs
total_training_steps = len(train_dataloader) * epochs

# Se agrega un scheduler para reducir linealmente el learning rate
# a lo largo de las epochs.
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_training_steps)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

Se configura BERT para NER (Named Entity Recognition)

In [54]:
# pip install seqeval
from seqeval.metrics import f1_score, accuracy_score

# Se guarda el costo promedio luego de cada epoch para plotearla
train_loss_values, validation_loss_values = [], []

# Barra de progreso de TQDM incluida dentro del for
for _ in trange(epochs, desc="Epoch"):

    # =================================
    # ENTRENAMIENTO
    # =================================

    # Se coloca el modelo en "training mode"
    model.train()

    # Se resetea el costo para la epoch actual
    total_loss = 0

    # Loop de entrenamiento
    for step, batch in enumerate(train_dataloader):

        # Se agrega el batch actual a la GPU
        batch = tuple(t.to(device) for t in batch)

        # Se extraen los IDs, input mask y labels del batch
        batch_ids, batch_mask, batch_labels = batch

        # Se limpian los gradientes calculados
        model.zero_grad()

        # Forward prop
        outputs = model(batch_ids, token_type_ids=None, attention_mask=batch_mask, labels=batch_labels)

        # Se obtiene el costo de la salida
        loss = outputs[0]

        # Backward prop
        loss.backward()

        # Se toma nota del costo de entrenamiento
        total_loss += loss.item()

        # Se evitan "exploding gradients" al cortar la normal del gradiente
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)

        # Se actualizan los parámetros usando los gradientes
        optimizer.step()

        # Se actualiza el learning rate (decay)
        scheduler.step()

    # Se calcula el costo promedio de entrenamiento
    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Costo Promedio de Entrenamiento: {avg_train_loss}")

    # Se guarda el valor de costo para plotear la curva de aprendizaje
    train_loss_values.append(avg_train_loss)

    # =================================
    # VALIDACIÓN
    # =================================

    # Se mide el desempeño del modelo en el set de validación
    
    # Se setea el modelo en modo de evaluación
    model.eval()

    # Se reinicia el costo de validación
    val_loss, val_accuracy = 0, 0

    # Arrays para predicciones y labels
    pred_labels, true_labels = [], []

    # Por cada batch
    for batch in valid_dataloader:

        # Se agrega el batch actual a la GPU
        # Se extraen los IDs, input mask y labels del batch
        batch = tuple(t.to(device) for t in batch)
        batch_ids, batch_mask, batch_labels = batch

        # Se indica al modelo que no calcule o guarde gradientes para
        # ahorrar memoria y acelerar la validación.
        with torch.no_grad():

            # Forward prop.
            # Esto va a retornar logits en lugar de costo debido a que no se proveyeron labels
            outputs = model(batch_ids, token_type_ids=None, attention_mask=batch_mask, labels=batch_labels)

        # Se mueven los logits y labels al CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = batch_labels.to("cpu").numpy()

        # Se calcula la media del costo
        val_loss += outputs[0].mean().item()

        # Se añaden las labels predichas y las reales a la lista "global"
        pred_labels.extend([list(pred) for pred in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    # Costo de validación promedio de epoch
    val_loss = val_loss / len(valid_dataloader)
    validation_loss_values.append(val_loss)
    print(f"Validation Loss: {val_loss}")
    
    # Se obtienen las tags predichas y las reales
    # Se ignoran las predicciones que corresponden a tags de padding
    pred_tags  = [unique_tags[p_i] for p,l in zip(pred_labels, true_labels)
                                  for p_i, l_i in zip(p, l) if unique_tags[l_i] != "PAD"]

    valid_tags = [unique_tags[l_i] for l   in zip(pred_labels, true_labels)
                                  for l_i in l if unique_tags[l_i] != "PAD"]

    # Se imprimen los scores
    print(f"Validation Accuracy: {accuracy_score(pred_tags, valid_tags)}")
    print(f"Validation F1-Score: {f1_score(pred_tags, valid_tags)}")
    print()

Epoch:   0%|          | 0/3 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 2.00 GiB total capacity; 1.13 GiB already allocated; 0 bytes free; 1.30 GiB reserved in total by PyTorch)

Se visualizan los costos de entrenamiento y de validación

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Se utiliza el estilo de plot de seaborn
sns.set(style="darkgrid")

# Se incrementa el tamaño de plot y el tamaño de letra
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Se plotean las curvas de aprendizaje
plt.plot(train_loss_values, 'b-o', label="Training Loss")
plt.plot(validation_loss_values, 'b-o', label="Validation Loss")
plt.title("Learning Curves")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

Se realiza una predicción en una nueva oración

In [ ]:
test_sentence = """
Mr. Trump’s tweets began just moments after a Fox News report by Mike Tobin, a 
reporter for the network, about protests in Minnesota and elsewhere. 
"""

# Se tokeniza el texto y se carga en la GPU
tokenized_sentence = tokenizer.encode(test_sentence)
input_ids = torch.tensor([tokenized_sentence]).cuda()

# Se procesa la oración con el modelo (el modelo no debe de actualizar gradientes)
with torch.no_grad():
    output = model(input_ids)

# Se obtienen los indices para las labels
label_indices = np.argmax(output[0].to("cpu").numpy(), axis=2)

# Se vuelven a unir los tokens separados durante el procesamiento
tokens = tokenizer.convert_ids_to_tokens(input_ids.to("cpu").numpy()[0])

# Listas para los nuevos tokens y labels luego de "re-unir" los 
# tokens spliteados.
new_tokens, new_labels = [], []

# Si los tokens fueron separados (Inician con ##), se suman los 
# caracteres distintos de "##" al "token base". Si no inicia con
# esos caracteres, entonces simplemente se agregan al registro.
for token, label_idx in zip(tokens, label_indices[0]):
    
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(unique_tags[label_idx])
        new_tokens.append(token)

# Se imprime la clasificación de la oración
for token, label in zip(new_tokens, new_labels):
    print(f"{label}\t{token}")
